# test note


* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
from pathlib import Path

# settings cell

# mounted dir
ait_dir = Path('/workdir/root')

ait_name='eval_ca_distribution'
ait_full_name='eval_ca_distribution_0.1'

# (dockerホスト側の)インベントリ登録用アセット格納ルートフォルダ
#invenotory_root_dir=r'F:\qai-testbed\dev\qai-testbed\ait_repository\eval_ca_distribution_0.1\local_qai\inventory'
invenotory_root_dir=r'C:\eval_ca_distribution\local_qai\inventory'

# entry point address
# コンテナ起動かどうかでポート番号が変わるため、切り替える
is_container = True
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

# aitのデプロイフラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_ait = True
#is_init_ait = False

# インベントリの登録フラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_inventory = True
#is_init_inventory = False


In [2]:
import requests
from pathlib import Path
import zipfile
from glob import glob
import pprint
import json
import time

def get_bk(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{backend_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def get_ip(path:str, is_print_response:bool = True, is_print_json:bool = True):
    response = requests.get(f'{ip_entry_point}{path}')
    if is_print_response:
        print(response)
    if is_print_json:
        pprint.pprint(response.json())
    return response

def post_bk(path:str, data):
    response = requests.post(f'{backend_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response

def post_ip(path:str, data):
    response = requests.post(f'{ip_entry_point}{path}', 
                             json.dumps(data),
                             headers={'Content-Type': 'application/json'}) 
    print(response)  
    pprint.pprint(response.json())
    return response
    
def uploadzip_id(zip_path:str):
    fileobj = open(zip_path, 'rb')
    response = requests.post(f'{ip_entry_point}/deploy-dag', 
                             data = {"name":"dag_zip"}, 
                             files={"archive": (str(Path(zip_path).name), fileobj)})
    print(response)
    pprint.pprint(response.json())
    return response

def create_zip():
    zip_dir = ait_dir
    zip_file = zip_dir.parent / f'{ait_full_name}.zip'

    with zipfile.ZipFile(zip_file, 'w', compression=zipfile.ZIP_STORED) as new_zip:
        new_zip.write(str(zip_dir.joinpath('dag.py')), arcname=f'{ait_full_name}/dag.py')
        new_zip.write(str(zip_dir.joinpath('dev').joinpath('dockerfile')), arcname=f'{ait_full_name}/dev/dockerfile')
        
        dir_path = zip_dir.joinpath('dev').joinpath('repository')
        files = glob(str(dir_path.joinpath('**')), recursive=True)
        for file in files:
            file_name = Path(file).name
            new_zip.write(file, arcname=f'{ait_full_name}/dev/repository/{file_name}')
    return zip_file

def async_build_ait(zip_path):
    zip_name = str(Path(zip_path).name)
    with open(zip_path, 'rb') as file_obj:
        response = requests.post(f'{ip_entry_point}/async-deploy-dag', 
                                 files={"dag_zip": (zip_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response
    
def post_manifest(manifest_path):
    manifest_name = str(Path(manifest_path).name)
    with open(manifest_path, 'rb') as file_obj:
        response = requests.post(f'{backend_entry_point}/testRunners/manifest', 
                                 files={"ait.manifest": (manifest_name, file_obj)})
    print(response)
    pprint.pprint(response.json())
    return response

def _find_file(dir_path: Path, file_name: str):
    files = glob(str(dir_path.joinpath(f'**/{file_name}')), recursive=True)
    if len(files) == 0:
        raise Exception(f'not found {file_name} in zip.')
    elif len(files) > 1:
        raise Exception(f'{file_name} must be one exists in zip.')
    return files[0]

def deploy_ait_async(need_build_container:bool=True):    
    # add manifest
    post_manifest(_find_file(ait_dir, 'ait.manifest.json'))

    # deploy
    if need_build_container:
        zip_file = create_zip()
        async_build_ait(zip_file)

def wait_deploy_complete(wait_time:float=1):
    # animation = "|/-\\"
    animation = [
        " [*     ]",
        " [ *    ]",
        " [  *   ]",
        " [   *  ]",
        " [    * ]",
        " [     *]",
        " [    * ]",
        " [   *  ]",
        " [  *   ]",
        " [ *    ]",
    ]
    i = 0

    while True:
        print(animation[i % len(animation)], end="\r")
        i += 1

        json = get_ip('/async-deploy-dag', is_print_response=False, is_print_json=False).json()
        if json['Code'] != 'D00011':
            if json['Code'] != 'D00010':
                # D00010以外はエラー発生のため、コンソールに出力
                pprint.pprint(json)
            else:
                print('complete')
            break
        time.sleep(wait_time)

def wait_run_complete(wait_time:float=1):
    # animation = "|/-\\"
    animation = [
        " [*     ]",
        " [ *    ]",
        " [  *   ]",
        " [   *  ]",
        " [    * ]",
        " [     *]",
        " [    * ]",
        " [   *  ]",
        " [  *   ]",
        " [ *    ]",
    ]
    i = 0

    while True:
        print(animation[i % len(animation)], end="\r")
        i += 1

        run_status = get_bk('/dep-a/projects/1/testDescriotions/run-status', is_print_response=False, is_print_json=False).json()
        if run_status['Job']['Status'] != 'RUNNING':
            pprint.pprint(run_status['Runs'])
            break
        time.sleep(wait_time)


In [3]:
# health check

get_bk('/health-check')
get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [4]:
# deploy AIT
if is_init_ait:
    deploy_ait_async(need_build_container=True)
    wait_deploy_complete(wait_time=0.5)
else:
    print('skip deploy AIT')

<Response [200]>
{'Code': 'M00001', 'Message': 'Add AIT manifest success'}
<Response [200]>
{'Code': 'D00001', 'Message': 'Deploy success'}
complete]


In [5]:
# add inventories

if is_init_inventory:
    post_bk('/dep-a/projects/1/inventories', {
        'Name': 'trained_model_checkpoint',
        'Type': "model", 'FileSystem': "UNIX_FILE_SYSTEM",
        'Address': f'{invenotory_root_dir}\\trained_model_checkpoint\\trained_model_checkpoint.zip',
        'Description': "学習したモデル_checkpoint",
        'Formats': ["zip"],
    })
    post_bk('/dep-a/projects/1/inventories', {
        'Name': 'trained_model_graph',
        'Type': "model", 'FileSystem': "UNIX_FILE_SYSTEM",
        'Address': f'{invenotory_root_dir}\\trained_model_graph\\trained_model_graph.zip',
        'Description': "学習したモデル_graph",
        'Formats': ["zip"],
    })
    post_bk('/dep-a/projects/1/inventories', {
        'Name': 'trained_model_protobuf',
        'Type': "model", 'FileSystem': "UNIX_FILE_SYSTEM",
        'Address': f'{invenotory_root_dir}\\trained_model_protobuf\\trained_model_protobuf.zip',
        'Description': "学習したモデル_protobuf",
        'Formats': ["zip"],
    })
    post_bk('/dep-a/projects/1/inventories', {
        'Name': 'test_set_images',
        'Type': "dataset", 'FileSystem': "UNIX_FILE_SYSTEM",
        'Address': f'{invenotory_root_dir}\\test_set_images\\test_set_images.zip',
        'Description': "テスト画像セット（bdd100K）",
        'Formats': ["zip"],
    })
    post_bk('/dep-a/projects/1/inventories', {
        'Name': 'test_set_labels',
        'Type': "dataset", 'FileSystem': "UNIX_FILE_SYSTEM",
        'Address': f'{invenotory_root_dir}\\test_set_labels\\bdd100k_labels_images_val.json',
        'Description': "テスト画像ラベル（bdd100K）",
        'Formats': ["json"],
    })
    post_bk('/dep-a/projects/1/inventories', {
        'Name': 'labels_define',
        'Type': "dataset", 'FileSystem': "UNIX_FILE_SYSTEM",
        'Address': f'{invenotory_root_dir}\\labels_define\\mscoco_complete_label_map.pbtxt',
        'Description': "ラベル定義",
        'Formats': ["txt"],
    })
else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [6]:
# get ait_json and inventory_jsons

res_json = get_bk('/QualityMeasurements/RelationalOperators', is_print_json=False).json()
eq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '=='][0])
nq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '!='][0])
gt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>'][0])
ge_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>='][0])
lt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<'][0])
le_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<='][0])

res_json = get_bk('/testRunners', is_print_json=False).json()
res_json

<Response [200]>
<Response [200]>


{'Result': {'Code': 'I52000', 'Message': 'get test runners success.'},
 'TestRunners': [{'Id': 2,
   'QualityDimensionId': 5,
   'Name': 'eval_ca_distribution',
   'Description': '画像分類モデルに画像データ（.jpg）を推論させ、\r\n推論結果を正解データ（.json）と比較し、\r\n推論結果がどの程度正確かをファイル出力する。',
   'Author': 'AIST',
   'Version': '0.1',
   'Quality': 'https://airc.aist.go.jp/aiqm/quality/internal/機械学習モデルの正確性',
   'LandingPage': '',
   'Reference': [],
   'ParamTemplates': [],
   'TargetInventories': [{'Id': 2,
     'Name': 'trained_model_checkpoint',
     'Type': 'model',
     'Description': '学習したモデル_checkpoint',
     'Formats': [{'Id': 7, 'Format': 'zip'}],
     'Schema': 'https://www.tensorflow.org/guide/saved_model'},
    {'Id': 3,
     'Name': 'trained_model_graph',
     'Type': 'model',
     'Description': '学習したモデル_graph',
     'Formats': [{'Id': 7, 'Format': 'zip'}],
     'Schema': 'https://www.tensorflow.org/guide/saved_model'},
    {'Id': 4,
     'Name': 'trained_model_protobuf',
     'Type': 'model',
     'Descri

In [7]:
ait_json = [j for j in res_json['TestRunners'] if j['Name'] == ait_name][-1]

res_json = get_bk('/dep-a/projects/1/inventories', is_print_json=False).json()
inv_1_json = [j for j in res_json['Inventories'] if j['Name'] == 'trained_model_checkpoint'][-1]
inv_2_json = [j for j in res_json['Inventories'] if j['Name'] == 'trained_model_graph'][-1]
inv_3_json = [j for j in res_json['Inventories'] if j['Name'] == 'trained_model_protobuf'][-1]
inv_4_json = [j for j in res_json['Inventories'] if j['Name'] == 'test_set_images'][-1]
inv_5_json = [j for j in res_json['Inventories'] if j['Name'] == 'test_set_labels'][-1]
inv_6_json = [j for j in res_json['Inventories'] if j['Name'] == 'labels_define'][-1]

<Response [200]>


In [8]:
inv_1_json

{'Id': 2,
 'Name': 'trained_model_checkpoint',
 'Type': 'model',
 'FileSystem': 'UNIX_FILE_SYSTEM',
 'Address': 'C:\\eval_ca_distribution\\local_qai\\inventory\\trained_model_checkpoint\\trained_model_checkpoint.zip',
 'Description': '学習したモデル_checkpoint',
 'Formats': [{'Id': 7, 'Format': 'zip'}]}

In [9]:
ait_json['TargetInventories'][0]

{'Id': 2,
 'Name': 'trained_model_checkpoint',
 'Type': 'model',
 'Description': '学習したモデル_checkpoint',
 'Formats': [{'Id': 7, 'Format': 'zip'}],
 'Schema': 'https://www.tensorflow.org/guide/saved_model'}

In [10]:
# add teast_descriptions

post_bk('/dep-a/projects/1/testDescriotions', {
#    "Name": "eval_ca_distribution_0.1",
    "Name": "eval_ca_distribution",
    "QualityDimensionID": 5,
    "QualityMeasurements": [
        {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][4]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][5]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][6]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][7]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][8]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id},
        {"Id":ait_json['Report']['Measures'][9]['Id'], "Value":"0.5", "RelationalOperatorId":gt_id}
    ],
    "TargetInventories": [
        {"Id":1, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
        {"Id":2, "InventoryId": inv_2_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']},
        {"Id":3, "InventoryId": inv_3_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][2]['Id']},
        {"Id":4, "InventoryId": inv_4_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][3]['Id']},
        {"Id":5, "InventoryId": inv_5_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][4]['Id']},
        {"Id":6, "InventoryId": inv_6_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][5]['Id']}
    ],
    "TestRunner": {
        "Id":ait_json['Id'],
        "Params":[

        ]
    }
})

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


<Response [200]>

In [11]:
# get test_description_jsons

res_json = get_bk('/dep-a/projects/1/testDescriotions', is_print_json=False).json()
# td_1_json = [j for j in res_json['Test']['TestDescriptions'] if j['Name'] == 'eval_ca_distribution_0.1'][-1]
td_1_json = [j for j in res_json['Test']['TestDescriptions'] if j['Name'] == 'eval_ca_distribution'][-1]
td_1_json


<Response [200]>


{'Id': 2,
 'Name': 'eval_ca_distribution',
 'Result': 'NA',
 'ResultDetail': 'NA',
 'CreationDatetime': '2020-09-25T00:25:20.705133',
 'TargetInventories': [{'Id': 2,
   'Name': 'trained_model_checkpoint',
   'Type': 'model',
   'Description': '学習したモデル_checkpoint',
   'TemplateInventoryId': 2},
  {'Id': 3,
   'Name': 'trained_model_graph',
   'Type': 'model',
   'Description': '学習したモデル_graph',
   'TemplateInventoryId': 3},
  {'Id': 4,
   'Name': 'trained_model_protobuf',
   'Type': 'model',
   'Description': '学習したモデル_protobuf',
   'TemplateInventoryId': 4},
  {'Id': 5,
   'Name': 'test_set_images',
   'Type': 'dataset',
   'Description': 'テスト画像セット（bdd100K）',
   'TemplateInventoryId': 5},
  {'Id': 6,
   'Name': 'test_set_labels',
   'Type': 'dataset',
   'Description': 'テスト画像ラベル（bdd100K）',
   'TemplateInventoryId': 6},
  {'Id': 7,
   'Name': 'labels_define',
   'Type': 'dataset',
   'Description': 'ラベル定義',
   'TemplateInventoryId': 7}]}

In [12]:
# run test_descriptions

post_bk('/dep-a/projects/1/testDescriotions/runners', {
    "Command": "AsyncStart",
    "TestDescriptionIds": [td_1_json['Id']]
})

wait_run_complete(wait_time=0.5)

<Response [200]>
{'Job': {'Id': '1', 'StartDateTime': '2020-09-25 09:25:26.813977+09:00'},
 'Result': {'Code': 'R12000', 'Message': 'job launch success.'}}
[{'Id': 1,
  'Result': 'NG',
  'ResultDetail': '{\n'
                  '  "traffic_sign_accuracy": false,\n'
                  '  "traffic_light_accuracy": false,\n'
                  '  "car_accuracy": false,\n'
                  '  "rider_accuracy": false,\n'
                  '  "motor_accuracy": false,\n'
                  '  "person_accuracy": false,\n'
                  '  "bus_accuracy": false,\n'
                  '  "truck_accuracy": false,\n'
                  '  "bike_accuracy": false,\n'
                  '  "train_accuracy": false\n'
                  '}',
  'Status': 'DONE',
  'TestDescriptionID': 2}]


In [13]:
res_json = get_bk('/dep-a/projects/1/testDescriotions/{}'.format(td_1_json['Id']), is_print_json=False).json()
pprint.pprint(res_json)

<Response [200]>
{'Result': {'Code': 'T32000', 'Message': 'get detail success.'},
 'TestDescriptionDetail': {'Id': 2,
                           'Name': 'eval_ca_distribution',
                           'Opinion': '',
                           'QualityDimension': {'Id': 5, 'Name': '機械学習モデルの正確性'},
                           'QualityMeasurements': [{'Description': 'accuracy '
                                                                   'predicted '
                                                                   'of '
                                                                   'traffic_sign',
                                                    'Id': 2,
                                                    'Name': 'traffic_sign_accuracy',
                                                    'RelationalOperatorId': 3,
                                                    'Structure': 'single',
                                                    'Value': '0.5'},
                

In [14]:
# generate report
res = post_bk('/dep-a/projects/1/testDescriotions/reportGenerator', {
    "Command": "Generate",
    "Destination": [str(td_1_json['Id'])]
})
pprint.pprint(res.json)

<Response [200]>
{'OutParams': {'ReportUrl': 'http://127.0.0.1:8888/qai-testbed/api/0.0.1/download/19'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
<bound method Response.json of <Response [200]>>
